In [ ]:
import nd2reader
import numcodecs
import numpy as np
import skimage
import zarr
from numcodecs import Blosc, Delta

%matplotlib inline
from copy import deepcopy

import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook

In [ ]:
frames = nd2reader.ND2Reader(
    "/home/jqs1/scratch/fidelity/171018/20171018_TrxnError_ID.nd2"
)

In [ ]:
frames2 = nd2reader.ND2Reader(
    "/home/jqs1/scratch/fidelity/171018/20171018_TrxnError.nd2"
)

In [ ]:
# z = zarr.open_array('/home/jqs1/scratch/fidelity/test/171018_id.zarr', mode='r+')
z2 = zarr.open_array("/home/jqs1/scratch/fidelity/test/171018.zarr", mode="r+")

In [ ]:
frames2.sizes

In [ ]:
v = 20
ary = np.dstack(
    [frames2.get_frame_2D(c=0, t=t, v=v) for t in tnrange(frames2.sizes["t"])]
)

In [ ]:
v = 20
ary2 = np.dstack(
    [
        np.dstack(
            [frames2.get_frame_2D(c=0, t=t, v=v) for t in tnrange(frames2.sizes["t"])]
        )[:, :, np.newaxis, :]
        for c in tnrange(frames2.sizes["c"])
    ]
)

In [ ]:
def bench_composite(channel_stack):
    # channel_imgs = [frames.get_frame_2D(c=i, t=t, v=v) for i in range(n_channels)]
    scaled_imgs = [
        channel_imgs[i][:, :, np.newaxis] / np.percentile(channel_imgs[i], 99.9)
        for i in range(n_channels)
    ]
    for scaled_img in scaled_imgs:
        np.clip(scaled_img, 0, 1, scaled_img)  # clip in place
    colored_imgs = [scaled_imgs[i] * np.array(colors[i]) for i in range(n_channels)]
    imgs_to_combine = [colored_imgs[i] for i in range(n_channels)]
    if not len(imgs_to_combine):
        imgs_to_combine = [np.ones(colored_imgs[0].shape)]  # white placeholder
    img = imgs_to_combine[0]
    for img2 in imgs_to_combine[1:]:
        img = 1 - (1 - img) * (1 - img2)
    # return img

In [ ]:
def bench_segment(channel_stack):
    pass

In [ ]:
scale = 2**6
plt.figure(figsize=(20, 20))
plt.imshow(ary[:, :, 0] // scale)
plt.figure(figsize=(20, 20))
plt.plot(ary[1000, :, 0] // scale)
plt.figure(figsize=(20, 20))
plt.plot(ary[:, 1000, 0] // scale)

In [ ]:
%%time
# filters = [numcodecs.Delta(dtype='u2', astype='u2')]
# filters = [numcodecs.Quantize(4, dtype='u2', astype='u2')]
# filters = [numcodecs.FixedScaleOffset(ary.mean(), ary.std(), dtype='u2', astype='u2')]
filters = []
compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE)  # , blocksize=0)
z2 = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/z2.zarr",
    mode="w",
    shape=ary.shape,
    chunks=(512, 512, 1),
    filters=filters,
    dtype="u2",
    order="C",
    compressor=compressor,
)
z2[:] = ary // 64
display(z2.info)

In [ ]:
%%time
# filters = [numcodecs.Delta(dtype='u2', astype='u2')]
# filters = [numcodecs.Quantize(4, dtype='u2', astype='u2')]
# filters = [numcodecs.FixedScaleOffset(ary.mean(), ary.std(), dtype='u2', astype='u2')]
filters = []
compressor = Blosc(cname="zstd", clevel=1, shuffle=Blosc.SHUFFLE)  # , blocksize=0)
z3 = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/z3.zarr",
    mode="w",
    shape=ary.shape,
    chunks=(512, 512, 1),
    filters=filters,
    dtype="u2",
    order="C",
    compressor=compressor,
)
z3[:] = ary // 64
display(z3.info)

In [ ]:
%%time
# filters = [numcodecs.Delta(dtype='u2', astype='u2')]
# filters = [numcodecs.Quantize(4, dtype='u2', astype='u2')]
# filters = [numcodecs.FixedScaleOffset(ary.mean(), ary.std(), dtype='u2', astype='u2')]
filters = []
compressor = Blosc(cname="blosclz", clevel=1, shuffle=Blosc.SHUFFLE)  # , blocksize=0)
z3 = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/z3.zarr",
    mode="w",
    shape=ary.shape,
    chunks=(512, 512, 1),
    filters=filters,
    dtype="u2",
    order="C",
    compressor=compressor,
)
z3[:] = ary // 64
display(z3.info)

In [ ]:
z1 = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/z2.zarr",
    mode="w",
    shape=ary3.shape,
    chunks=(1024, 1024, 1, 10),
    dtype="i2",
)

In [ ]:
%%time
z1[:] = ary2

In [ ]:
z1.info

In [ ]:
%%time
filters = []  # [Delta(dtype='i2')]
compressor = Blosc(cname="zstd", clevel=2, shuffle=Blosc.SHUFFLE, blocksize=0)
z2 = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/z2.zarr",
    mode="w",
    shape=ary2.shape,
    chunks=(1024, 1024, 1, 10),
    dtype="i2",
    filters=filters,
    order="F",
    compressor=compressor,
)
z2[:] = ary2

In [ ]:
z2.info

In [ ]:
%%time
bench_composite(z1[:, :, :, 0])

In [ ]:
%%time
for t in range(frames2.sizes["t"]):
    np.percentile(z1[:, :, 0, t], 99.9)

In [ ]:
%%time
for t in range(frames2.sizes["t"]):
    np.percentile(z2[:, :, 0, t], 99.9)

In [ ]:
%%time
for t in range(frames2.sizes["t"]):
    np.percentile(z[:, :, 0, t], 99.9)

In [ ]:
%%time
for t in range(frames2.sizes["t"]):
    np.percentile(ary2[:, :, 0, t], 99.9)

In [ ]:
%%time
filters = []  # [Delta(dtype=np.uint16, astype=np.int16)]
compressor = Blosc(cname="zstd", clevel=1, shuffle=Blosc.SHUFFLE, blocksize=0)
z = zarr.array(
    ary2, chunks=(None, None, 1, 5), filters=filters, order="C", compressor=compressor
)
print(z.info)

In [ ]:
len(set(ary[0, 0, :])) / len(ary[0, 0, :].flat)

In [ ]:
plt.plot(ary[0, 0:10, :].T)

In [ ]:
plt.plot(ary[0:10, 0, :].T)

In [ ]:
aryb = np.rollaxis(ary, 2, 0)

In [ ]:
%%time
filters = []  # [Delta(dtype=np.uint16, astype=np.int16)]
compressor = Blosc(cname="zstd", clevel=9, shuffle=Blosc.SHUFFLE, blocksize=0)
# compressor = Blosc(cname='blosclz', clevel=9, shuffle=Blosc.BITSHUFFLE, blocksize=0)
z = zarr.array(
    aryb, chunks=(1, 512, 512), filters=filters, order="C", compressor=compressor
)
print(z.info)

In [ ]:
frames.filename

In [ ]:
compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)
frames_z = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/171018_id.zarr",
    mode="w",
    shape=[frames.sizes[n] for n in "vctyx"],
    chunks=(1, 1, 1, 512, 512),
    dtype="u2",
    order="C",
    compressor=compressor,
)
meta = deepcopy(frames.metadata)
meta["date"] = meta["date"].isoformat()
frames_z.attrs["metadata"] = meta
for v in tnrange(frames.sizes["v"]):
    for t in range(frames.sizes["t"]):
        for c in range(frames.sizes["c"]):
            frames_z[v, c, t, :, :] = frames.get_frame_2D(c=c, t=t, v=v)

In [ ]:
compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)
frames2_z = zarr.open_array(
    "/home/jqs1/scratch/fidelity/test/171018.zarr",
    mode="w",
    shape=[frames2.sizes[n] for n in "vctyx"],
    chunks=(1, 1, 1, 512, 512),
    dtype="u2",
    order="C",
    compressor=compressor,
)
# for v in tnrange(frames2.sizes['v']):
meta = deepcopy(frames2.metadata)
meta["date"] = meta["date"].isoformat()
frames2_z.attrs["metadata"] = meta
for v in tnrange(200):
    for t in range(frames2.sizes["t"]):
        for c in range(frames2.sizes["c"]):
            frames2_z[v, c, t, :, :] = frames2.get_frame_2D(c=c, t=t, v=v)

In [ ]:
numcodecs.

In [ ]:
compressor = numcodecs.BZ2

In [ ]:
a = ary - ary.mean(axis=(0, 1)).astype(np.uint16)

In [ ]:
b = np.rollaxis(a, 2, 0)
print(a.shape, b.shape)

In [ ]:
c = np.rollaxis(a, 2, 1)
print(a.shape, b.shape)

In [ ]:
%%time
filters = []  # [Delta(dtype=np.uint16, astype=np.int16)]
compressor = Blosc(cname="zstd", clevel=3, shuffle=Blosc.SHUFFLE, blocksize=0)
z = zarr.array(
    c, chunks=(256, 20, 256), filters=filters, order="C", compressor=compressor
)
print(z.info)

In [ ]:
%%time
filters = []  # [Delta(dtype=np.uint16, astype=np.int16)]
compressor = Blosc(cname="zstd", clevel=3, shuffle=Blosc.SHUFFLE, blocksize=0)
z = zarr.array(
    b, chunks=(None, 256, 256), filters=filters, order="C", compressor=compressor
)
print(z.info)

In [ ]:
%%time
filters = []  # [Delta(dtype=np.uint16, astype=np.int16)]
compressor = Blosc(cname="zstd", clevel=3, shuffle=Blosc.SHUFFLE, blocksize=0)
z = zarr.array(
    a, chunks=(256, 256, 20), filters=filters, order="F", compressor=compressor
)
print(z.info)